# Galaxy integration through API from FAIRDOM

* Upload data from list of URLs
* Run workflow


In [ ]:
from bioblend import galaxy
from collections import OrderedDict
import json
import time
import re
f= open("dakey.txt","r")
assert(f)
galaxy_api_key = f.readline()
f.close()
galaxy_api_key = galaxy_api_key.rstrip()

In [ ]:
f= open("JSON_INPUT","r")
#f= open("input.json","r")
json_input = json.load(f)
f.close()
json_input = json_input["marked"]
json_input

In [ ]:
def extract_url(s):
    e = re.compile(".*href=\"(.*)\".*");
    m = e.match(s)
    if(m):
        return_value = m.group(1)
        return return_value
    else:
        return s
    
r = extract_url('<a target="_blank" href="ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR105/002/ERR1059392/ERR1059392_1.fastq.gz">ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR105/002/ERR1059392/ERR1059392_1.fastq.gz</a>')
r

In [ ]:
samples = dict()

for i in json_input.values():
    for j in range(1,len(json_input['0']["values"])):
        sample_name = json_input['0']["values"][j]
        step_name = json_input['1']["values"][j]
        url = extract_url(json_input['2']["values"][j])
        
        if (not(sample_name in samples.keys())):
            samples[sample_name] = dict();
        
        samples[sample_name][step_name] = url
    
samples

In [ ]:
galaxy_config = {
    'url': 'https://usegalaxy.be',
    'api_key': galaxy_api_key # make an account on the Galaxy instance and then User - Preferences - Manage API key
}

In [ ]:
fairdom_config = {
    'investigation' : '8-way RIL population', # Wolfgang
    #'investigation' : '8-way_RIL_population',# Frederik
    
    #'study' : 'RIL_8-way_growth_chamber',    
    #'assay' : 'RNA_seq_E-MTAB-3965',         
    'workflow' : 'Salmon_maize_paired' # name of the workflow, assumes there is only one with that name
}

#samples = {
#    'RIL1' : 
#        { 
#            'forward' : 'ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR105/002/ERR1059392/ERR1059392_1.fastq.gz',
#            'reverse' : 'ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR105/002/ERR1059392/ERR1059392_2.fastq.gz',
#            #'phenotyping' : 'https://floppy.psb.ugent.be/index.php/s/0ZK7y1gZb26BAEg/download'
# }
#}

In [ ]:
print("The list of URLs read from the data file:\n")

for x in samples:
    print (x)
    for y in samples[x]:
        print (y +':'+ samples[x][y])

## Connecting to Galaxy

* Connection to Galaxy
'url': 'https://usegalaxy.be',
'api_key': 'API_KEY' # make an account on the Galaxy instance and then User - Preferences - Manage API key

* Info needed from FAIRDOM
investigation : 8-way RIL population
workflow : Salmon_maize_paired

* Info on data 

In [ ]:
gi = galaxy.GalaxyInstance(url=galaxy_config['url'], key=galaxy_config['api_key'])

In [ ]:

current_user = None

success = False;
while(not success):
    try:
        print ("Current user's information:\n")
        current_user = gi.users.get_current_user()

        print ("username:" + current_user["username"])
        print ("email:" + current_user["email"])
        print ("is_admin:" + str(current_user["is_admin"]))
        success = True
    except: 
        print("Failed: ")
        time.sleep(60)
        success = False

## Get the data and copy it to Galaxy

* I created a FAIRDOM data library where we will upload data, getting a link to it
* I need give an accouynt access to allow to write in this folder (feature not a bug)

In [ ]:

#I created a FAIRDOM data library where we will upload data, getting a link to it
#I need give an accouynt access to allow to write in this folder (feature not a bug)

success = False

while(not success):
    try:
        library = gi.libraries.get_libraries(name = 'FAIRDOM')
        success = True
    except: 
        print("Failed: ")
        time.sleep(60)
        success = False

In [ ]:
# If a folder with the investigation name already exist, 

success = False

while(not success):
    try:
        root_folder = gi.libraries.get_folders(library[0]['id'])
        files = gi.libraries.show_library(library[0]['id'], contents=True)

        investigation_present = False

        investigation_library = ""

        for file in files:
            if file['name'] == ("/" + fairdom_config['investigation']):
                investigation_present = True
                investigation_library = file['id']

        if not investigation_present:
            print("A folder with the investigation name doesn't exist, create investigation!")
            investigation_library =  gi.libraries.create_folder(library[0]['id'], fairdom_config['investigation'], description=None)[0]
        else:
            investigation_library =  gi.libraries.get_folders(library[0]['id'], name = "/" + fairdom_config['investigation'])[0]
            print("A folder with the investigation name already exist!")
        success = True

    except: 
        print("Exception: ")
        time.sleep(60)
        success = False  


In [ ]:
uploads = {}


success = False

while(not success):
    try:
        for sample in samples:
            print(sample)
            uploads[sample] = []
            for key, file in samples[sample].items():
                    print(file)
                    print("\n")
                    # does not check if file is present
                    file_present = False
                    for avail_file in files:
                        if avail_file['name'] == ("/" + fairdom_config['investigation'] + "/" + file):
                            print("file found: ")
                            print(avail_file)
                            print("\n")
                            uploads[sample].append(avail_file) 
                            file_present = True
                            break
                    if not file_present :
                        # this gives url as filename, can be changed through update, not yet implemented
                        uploaded_file = gi.libraries.upload_file_from_url(library[0]['id'], 
                             file_url = file, 
                             folder_id=investigation_library['id'], 
                             file_type='fastqsanger.gz',
                             #file_type='auto',                                             
                             #dbkey='?'
                            )
                        uploads[sample].append(uploaded_file[0])
        success = True
    except: 
        print("Exception: ")
        time.sleep(60)
        success = False  

In [ ]:
# to be improved, now waiting for all samples to be uploaded
not_yet_ready = True
errors = False
while not_yet_ready:
    for sample in samples:
        print(sample)
        
        for upload in uploads[sample]:
            success = False
            while(not success):
                try:
                    print(gi.libraries.show_dataset(library[0]['id'], upload['id'])['state'])
                    if gi.libraries.show_dataset(library[0]['id'], upload['id'])['state'] == 'ok':
                        not_yet_ready = False
                        # update_library_dataset(library[0]['id'], name="name_to_which_it_needs_to_be_changed")
                    elif gi.libraries.show_dataset(library[0]['id'], upload['id'])['state'] == 'error':
                        not_yet_ready = False
                        errors = True
                    else: 
                        not_yet_ready = True
                    success = True
                except:
                    success = False
if not_yet_ready:
    print("Waiting for upload")
    time.sleep(60)            
print("The files are Ready !")

## Run workflow

In [ ]:
#assumes a workflow with that name is present for the user
success = False
workflows = None
while(not success):
    try:
        workflows = gi.workflows.get_workflows(name = fairdom_config['workflow'], published=True)
        workflow = workflows[0]
        success = True
        print("Got workflow %s" % fairdom_config['workflow'])
    except: 
        print("Exception: ")
        time.sleep(60)
        success = False     

In [ ]:
invoked_workflows = {}

for sample in samples:
    success = False
    while(not success):
        try:   
            success = True
            # assuming order forward - reverse and only for the first pair
            inputs = {}
            inputs[0] = { 'src':'ld', 'id':uploads[sample][0]['id'] }
            inputs[1] = { 'src':'ld', 'id':uploads[sample][1]['id'] }

            invoked_workflow = gi.workflows.invoke_workflow(workflow['id'], 
                                     inputs=inputs, 
                                     import_inputs_to_history=True, 
                                     history_name=sample)
            print("invoked workflow...\n")
            print("update time:"+invoked_workflow["update_time"])
            print("history id:"+invoked_workflow["history_id"])
            print("uuid:"+invoked_workflow["uuid"])
            print("state:"+invoked_workflow["state"])
            print("workflow id:"+invoked_workflow["workflow_id"])
            print("model_class:"+invoked_workflow["model_class"])
            print("id:"+invoked_workflow["id"])
            invoked_workflows[sample] = invoked_workflow
        except: 
            print("Exception: ")
            time.sleep(60)
            success = False  


In [ ]:
# needs to match annotation of workflow step labels
# assuming downloading one file per step
downloads = {
    'FastQC forward': {
        'type' : 'html_file',
        'filename_postfix' : 'fastqc_fw.zip' # zip file with a html (and other stuff) inside
    }, 
    'FastQC reverse' : {
        'type' : 'text_file', # essentially same output as previous one, but text version
        'filename_postfix' : 'fastqc_rev.txt'
    }, 
    'Salmon' : {
        'type' : 'output_quant',
        'filename_postfix' : 'counts.txt'
    }
}

print("Downloading files:\n")

for x in downloads:
    print (x)
    for y in downloads[x]:
        print (y +':'+ downloads[x][y])

In [ ]:
all_ready = False

while not all_ready:
    time.sleep(10)
    all_ready = True
    for sample in samples:
        print (sample)
        filename_prefix = fairdom_config['investigation'] + '_' + sample + '_' # to do: link with sample name, for now hardcoded
        #print ("filename_prefix:"+filename_prefix)
        #print (str(gi.workflows.show_invocation(invoked_workflows[sample]['workflow_id'], invoked_workflows[sample]['id'])['steps']))
        
        steps = None
        
        success = False
        while(not success):
            try:   
                success = True
                steps = gi.workflows.show_invocation(invoked_workflows[sample]['workflow_id'], invoked_workflows[sample]['id'])['steps']
            except:
                success = False
                time.sleep(10)
        
        for step in steps:
            print("steps: %s" % step['job_id'])
            if step['job_id']: # input does not have job_id
                try:
                    state = gi.jobs.get_state(step['job_id'])
                    print(state)
                    if state == 'ok':
                        # job finished
                        # all_ready = True
                        # do not need to set this. it is true
                        continue
                    else:
                        # not finished, so set to false
                        all_ready = False
                except: 
                    all_ready = False
                    time.sleep(10)
print("done")                

In [ ]:
for sample in samples:
    print("Sample: %s" % sample)
    steps = None
    success = False
    while(not success):
        try:
            success = True
            steps = gi.workflows.show_invocation(invoked_workflows[sample]['workflow_id'], invoked_workflows[sample]['id'])['steps']
        except:
            time.sleep(10)
            success = False
    for step in steps:
        print("Step: %s" % step)
        if step['workflow_step_label'] in downloads:
            print("Downloading step: %s" % step['workflow_step_label'])
            outputs = None
            success = False
            while(not success):
                try:
                    success = True
                    outputs = gi.jobs.show_job(step['job_id'])['outputs']
                except:
                    time.sleep(10)
                    success = False                
            for output in outputs:
                print("output %s" % output)
                if output == downloads[step['workflow_step_label']]['type']:
                        #print(step['workflow_step_label'])
                        #print(output)
                        print(filename_prefix + downloads[step['workflow_step_label']]['filename_postfix'])
                        #print (outputs[output]['id'])
                        #print (outputs[output]['src'])
                        #gi.datasets.download_dataset(outputs[output]['id'], file_path=filename_prefix + downloads[step['workflow_step_label']]['filename_postfix'], use_default_filename=False, maxwait=12000)
                        
                        success = False
                        while(not success):
                            try:
                                success = True
                                with open( filename_prefix + downloads[step['workflow_step_label']]['filename_postfix'], 'bw') as f:
                                    print("Writing outputid %s" % outputs[output]['id'])
                                    dataset = gi.datasets.download_dataset(outputs[output]['id'], use_default_filename=False, maxwait=12000)
                                    f.write(dataset)
                                    #print(dataset)
                                    #f.close()
                            except:
                                time.sleep(10)
                                success = False                

                            
print("DONE")

In [ ]:
import pandas
import time

In [ ]:
frame = pandas.read_csv('8-way RIL population_RIL1_counts.txt','\t')
frame[0:20]

In [ ]:
result = frame.sort_values(['TPM'], ascending=[0])
result[0:20]

In [ ]:
result = result[0:20]

In [ ]:
import matplotlib.pyplot as plt;
%matplotlib inline

figsize = [10,10] # size ~cm

p = result.plot(x='Name',y=['TPM','NumReads'],xticks=range(0,20),figsize=figsize)
p.set_xticklabels(result.Name,rotation=90)
q = result.plot(x='Name',y='TPM',xticks=range(0,20),figsize=figsize)
q.set_xticklabels(result.Name,rotation=90)


plt.show()
time.sleep(10)